In [1]:
import pandas as pd
import re

In [2]:
# Load CSV file
df = pd.read_csv('Peru_2019_AudioMoth_Data_Full.csv')
cols = df.columns

/var/folders/dg/3ns9ght557jdxwl4wn8qmybr0000gn/T/ipykernel_6602/1965858857.py:2: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Peru_2019_AudioMoth_Data_Full.csv')


In [3]:
df.shape

(98638, 20)

In [4]:
df.head()

,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_145649.WAV,GRABADOR-SDZG-AM-1,20190616_145649.WAV,1802240,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 14:56,2.346667,NaN,Recorded at 14:56:49 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
1,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_150000.WAV,GRABADOR-SDZG-AM-1,20190616_150000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:00,60.000469,NaN,Recorded at 15:00:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
2,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_151000.WAV,GRABADOR-SDZG-AM-1,20190616_151000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:10,60.000469,NaN,Recorded at 15:10:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:13-08:00,WAV,WAV,audio/x-wav
3,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_152000.WAV,GRABADOR-SDZG-AM-1,20190616_152000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:20,60.000469,NaN,Recorded at 15:20:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:15-08:00,WAV,WAV,audio/x-wav
4,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_153000.WAV,GRABADOR-SDZG-AM-1,20190616_153000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:30,60.000469,NaN,Recorded at 15:30:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:16-08:00,WAV,WAV,audio/x-wav


In [5]:
df.dtypes

AudioMothCode         object
AudioMothID           object
SourceFile            object
Directory             object
FileName              object
FileSize               int64
Encoding             float64
NumChannels          float64
SampleRate           float64
AvgBytesPerSec       float64
BitsPerSample        float64
StartDateTime         object
Duration             float64
Error                 object
Comment               object
Artist                object
FileCreateDate        object
FileType              object
FileTypeExtension     object
MIMEType              object
dtype: object

In [6]:
df.isna().mean()

AudioMothCode        0.000000
AudioMothID          0.218739
SourceFile           0.000000
Directory            0.000000
FileName             0.000000
FileSize             0.000000
Encoding             0.035686
NumChannels          0.035686
SampleRate           0.035686
AvgBytesPerSec       0.035686
BitsPerSample        0.035686
StartDateTime        0.193891
Duration             0.035686
Error                0.964314
Comment              0.035686
Artist               0.218739
FileCreateDate       0.000000
FileType             0.035686
FileTypeExtension    0.035686
MIMEType             0.035686
dtype: float64

The columns which require entries to conduct a stratified random sample are `AudioMothCode`, `Duration`, `FileSize`, `StartDateTime`, and `Comment`. Based on the mean null values calculated above, we notice that all necessary columns except `AudioMothCode` contain some level of null values.

For the purpose of this assignment, we need information on the `AudioMothCode` for each row data, either `Duration` or `FileSize` to determine whether a video clip is a minute long, and a value in either `StartDateTime` or `Comment` to find the hour of the day in which the sound was recorded.

We will drop rows which don't have an `AudioMothCode`, both `StartDateTime` and `Comment` (Strata Level 1) or both `Duration` and `FileSize` (Strata Level 2). For this dataset, no filtering is necessary for `AudioMothCode` as no null values are present, so the implementation of dropping null values for this column is a generalization for datasets where this is a necessary step.

In [7]:
# Drop rows that do not contain enough information.

# Strata Level 1
df.dropna(subset=['AudioMothCode'], how='all', inplace=True)
df.dropna(subset=['Duration', 'FileSize'], how='all', inplace=True)

# Strata Level 2
df.dropna(subset=['StartDateTime', 'Comment'], how='all', inplace=True)

All entries in `FileSize` are in bytes. We will convert them to megabytes by dividing each entry by 1000000.

In [8]:
df['FileSize'] = (df['FileSize'] / 10**6).round(decimals=1)

In [9]:
# Choose rows with successful recordings
df = df[(df['FileSize'] >= 46.1) | (df['Duration'] >= 60)].reset_index(drop=True)

Next, we will find the timestamps, which are located in either `StartDateTime` or `Comment`. The following function extracts the hour from each timestamp, starting with the `StartDateTime` entry. If a value is not found in `StartDateTime`, it is recovered using `Comment`.

In [10]:
def findHour(df):
    try:
        # Convert string timestamps to Datetime objects
        datetimes = pd.to_datetime(df['StartDateTime'])
        hours = pd.DataFrame(datetimes.dt.hour.fillna(-1))
        hours = hours.astype(int)
        missingEntries = list(hours[hours['StartDateTime'] == -1].index)

        # Fill in all the missing entries using the Comment column
        for idx in missingEntries:
            comment = df.iloc[idx]['Comment']
            time = re.findall(r'\b(\d{2}):(\d{2}):(\d{2})\b', comment)
            hours.iloc[idx]['StartDateTime'] = int(time[0][0])

        return hours['StartDateTime'].tolist()
    except:
        # Dataset is empty
        return []

In [11]:
# Create Hour column with the hour times
df['Hour'] = findHour(df)

In [12]:
# Find Audiomoths with 24 entries, at least 1 for each timestamp
grouped = pd.DataFrame(df.groupby('AudioMothCode')['Hour'].nunique()).reset_index()
validCodes = list(grouped[grouped['Hour'] == 24]['AudioMothCode'])


Now that we know all the successful Audiomoths, we can perform stratified random sampling for each successful Audiomoth and timestamp.

In [13]:
with open('stratified.csv', 'w') as f:
    f.write('AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType\n')
    for code in validCodes:
        
        subset = df[(df['AudioMothCode'] == code)]

        # Use ungrouped keys to perform stratified random sample for each Hour entry
        sampled = subset.groupby(by='Hour', group_keys=False).apply(lambda x: x.sample(n=1))

        # Write entry to new file
        for index, line in sampled.iterrows():
            entry = ','.join([str(line[col]) for col in cols])
            f.write(entry + '\n')

f.close()

In [14]:
# Load in CSV file with data obtained using stratified random sampling
df_strat = pd.read_csv('stratified.csv')

The CSV file created displays 24 entries for each successful Audiomoth, each entry representing a different hour of the day. As a result, we were able to obtain a dataset with 744 entries that is signficantly scaled down from our original dataset 98,638 entries.

In [15]:
df_strat

,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190701_000000.WAV,GRABADOR-SDZG-AM-1,20190701_000000.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,01.07.2019 00:00,60.000469,NaN,Recorded at 00:00:00 01/07/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 04:18:08-08:00,WAV,WAV,audio/x-wav
1,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190622_012000.WAV,GRABADOR-SDZG-AM-1,20190622_012000.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,22.06.2019 01:20,60.000469,NaN,Recorded at 01:20:00 22/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:37:38-08:00,WAV,WAV,audio/x-wav
2,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190624_023000.WAV,GRABADOR-SDZG-AM-1,20190624_023000.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,24.06.2019 02:30,60.000469,NaN,Recorded at 02:30:00 24/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:46:55-08:00,WAV,WAV,audio/x-wav
3,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190620_033000.WAV,GRABADOR-SDZG-AM-1,20190620_033000.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,20.06.2019 03:30,60.000469,NaN,Recorded at 03:30:00 20/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:29:03-08:00,WAV,WAV,audio/x-wav
4,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190617_045000.WAV,GRABADOR-SDZG-AM-1,20190617_045000.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,17.06.2019 04:50,60.000469,NaN,Recorded at 04:50:00 17/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:15:45-08:00,WAV,WAV,audio/x-wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,WWF-5,NaN,GRABADOR-WWF-5/5D166F68.WAV,GRABADOR-WWF-5,5D166F68.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,NaN,60.000250,NaN,Recorded at 19:50:00 28/06/2019 (UTC) by Audio...,NaN,2019:12:07 18:16:33-08:00,WAV,WAV,audio/x-wav
740,WWF-5,NaN,GRABADOR-WWF-5/5D152748.WAV,GRABADOR-WWF-5,5D152748.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,NaN,60.000250,NaN,Recorded at 20:30:00 27/06/2019 (UTC) by Audio...,NaN,2019:12:07 19:07:29-08:00,WAV,WAV,audio/x-wav
741,WWF-5,NaN,GRABADOR-WWF-5/5D1D20B0.WAV,GRABADOR-WWF-5,5D1D20B0.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,NaN,60.000250,NaN,Recorded at 21:40:00 03/07/2019 (UTC) by Audio...,NaN,2019:12:07 15:59:54-08:00,WAV,WAV,audio/x-wav
742,WWF-5,NaN,GRABADOR-WWF-5/5D266940.WAV,GRABADOR-WWF-5,5D266940.WAV,46.1,1.0,1.0,384000.0,768000.0,16.0,NaN,60.000250,NaN,Recorded at 22:40:00 10/07/2019 (UTC) by Audio...,NaN,2019:12:07 19:33:30-08:00,WAV,WAV,audio/x-wav
